In [11]:
class PUR:
  def __init__(self, quantile_low=0.10, quantile_high=0.90,Start_date='2011-10-07',initial_capital=100000,max_buy =1000000, max_sell=1000000, gas = 0.001, URL ='https://raw.githubusercontent.com/SciEcon/bitcoin_golden_litecoin_silver/main/PU%20ratio%20%26%20Trading/LTC/LTC_2022_05_31.csv'):
    self.low = quantile_low
    self.high = quantile_high
    self.start = Start_date
    self.initial_capital = initial_capital
    self.max_buy = max_buy
    self.max_sell = max_sell
    self.gas = gas
    self.URL = URL
  def portfolio(self):
    import pandas as pd 
    import numpy as np
    pd.options.display.float_format = "{:,.2f}".format
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    data=pd.read_csv(self.URL,index_col='Unnamed: 0')
    data.index=pd.to_datetime(data.index)
    data = data[data.index>=self.start]
    prices = data['PriceUSD']
    quantiles_low=[]
    quantiles_high=[]
    cashes = []
    litecoins = []
    holdings = []
    states = []
    cash = self.initial_capital
    litecoin = 0
    holding = 0
    state = "" 
    def buy(i, cash, litecoin, price):
      units = cash/(price*(1+self.gas))
      if units < 1:
        buy_units = 0
      else:
        if units>self.max_buy:
          buy_units = self.max_buy
        else:
          buy_units = units
        print('index %d: buy %f units at price %.3f, current cash %.3f, current litecoin %.3f' % (i, buy_units, price, cash, litecoin))
      cost = buy_units*price*(1+self.gas)
      cash -= cost
      litecoin += buy_units
      holding = litecoin*price
      return buy_units, cash, litecoin, holding
    def sell(i, cash, litecoin, price):
      if litecoin == 0:
        sell_units = 0
      else:
        if litecoin > self.max_sell:
          sell_units = self.max_sell
        else:
          sell_units = litecoin
        print('index %d: sell %.3f units at price %.3f, current cash %.3f, current litecoin %.3f' % (i, sell_units, price, cash, litecoin))
      litecoin -=sell_units
      revenue = sell_units*price*(1-self.gas)
      cash += revenue
      holding = litecoin*price
      return sell_units, cash, litecoin, holding 

    for i in range(0,data.shape[0]):
      quantile_low = data['PU Ratio'].iloc[0:i].quantile(self.low)
      quantile_high = data['PU Ratio'].iloc[0:i].quantile(self.high)
      PUR = data['PU Ratio'].iloc[i]
      price = prices[i]
      if PUR <= quantile_low:
        buy_units, cash, litecoin, holding = buy(i, cash, litecoin, price)
        if buy_units>0:
          state = 1
        else:
          state = 0  
      elif PUR >= quantile_high:
        sell_units, cash, litecoin, holding = sell(i, cash, litecoin, price)
        if sell_units>0:
          state = -1
        else:
          state = 0
      else:
        state = 0
        litecoin = litecoin
        cash = cash
        holding =litecoin*price ### holding updates when prices update even when there is no buy or sell actions

      states.append(state)
      cashes.append(cash)
      litecoins.append(litecoin)
      holdings.append(holding)
      quantiles_low.append(quantile_low)
      quantiles_high.append(quantile_high)

    data.loc[:,'state']=states
    data.loc[:,'cash']=cashes
    data.loc[:,'litecoin']=litecoins
    data.loc[:,'holding']=holdings
    data.loc[:,'quantiles_low']=quantiles_low
    data.loc[:,'quantiles_high']=quantiles_high
    data['total']=data['cash']+data['holding']
    data['roi_gross'] = (data['total']-self.initial_capital)/self.initial_capital*100
    data['roi_daily'] = data['total'].pct_change()
    data=data.fillna(0)
    data['sharpe'] = 0
    for i in range(0, data.shape[0]):
      if data['roi_daily'].iloc[0:i].std()==0:
        data['sharpe'].iloc[i]=0
      else:
        data['sharpe'].iloc[i]=(i**0.5)*np.mean(data['roi_daily'].iloc[0:i])/data['roi_daily'].iloc[0:i].std()
    data=data.fillna(0)
    return data
  def figure_signal(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name ='PU Ratio',
            x=data.index,
            y=data['PU Ratio'],
            line_color="blue",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Price in USD',
            x=data.index,
            y=data['PriceUSD'],
            line_color="black",
            line_width=1
        ),
        secondary_y=True,
    )
    fig.add_trace(
    
    go.Scatter(
        mode='lines',
        name='PU Ratio ' +str(self.low)+ ' Quantile',
        x=data.index,
        y=data.quantiles_low,
        line_color="green",
        line_width=1,
    ),
    secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='PU Ratio ' +str(self.high)+' Quantile',
            x=data.index,
            y=data.quantiles_high,
            line_color="red",
            line_width=1,
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(name="buy",mode="markers", x=data.index[data.state==1], y=data['PriceUSD'][data.state==1], marker_symbol=119,
                              marker_line_color="green", marker_color="green",
                              marker_line_width=2, marker_size=15,
    ),
    secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(name="sell",mode="markers", x=data.index[data.state==-1], y=data['PriceUSD'][data.state==-1], marker_symbol=120,
                              marker_line_color="red", marker_color="red",
                              marker_line_width=2, marker_size=15,
    ),
        secondary_y=True,
    )

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>PU Ratio</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Price in USD</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Buy and Sell Signals: PU Ratio"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return
  def figure_portfolio(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name ='Cash',
            x=data.index,
            y=data['cash'],
            line_color="red",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Holding',
            x=data.index,
            y=data['holding'],
            line_color="green",
            line_width=1
        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Total',
            x=data.index,
            y=data['total'],
            line_color="blue",
            line_width=1
        ),
        secondary_y=False,
    )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>USD</b>", secondary_y=False)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Portfolio Time Series: PU Ratio Automated Trading Strategies"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return 
  def figure_returns(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name ='Gross ROI',
              x=data.index,
              y=data['roi_gross'],
              line_color="blue",
              line_width=1,

          ),
        secondary_y=False,
      )
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name='Annualized Sharpe Ratio',
              x=data.index,
              y=data['sharpe'],
              line_color="black",
              line_width=1
          ),
        secondary_y=True,
      )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Percent</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Ratio</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Gross ROI and Annualized Sharpe Ratio: PU Ratio Automated Trading Strategy"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return


In [12]:
PUR= PUR(quantile_low=0.10,quantile_high=0.90,max_buy=100,max_sell=100)

In [13]:
data= PUR.portfolio()

index 637: buy 100.000000 units at price 2.276, current cash 100000.000, current litecoin 0.000
index 642: sell 100.000 units at price 2.726, current cash 99772.220, current litecoin 100.000
index 673: buy 100.000000 units at price 2.320, current cash 100044.551, current litecoin 0.000
index 675: buy 100.000000 units at price 2.310, current cash 99812.314, current litecoin 100.000
index 679: buy 100.000000 units at price 2.384, current cash 99581.083, current litecoin 200.000
index 688: buy 100.000000 units at price 2.300, current cash 99342.427, current litecoin 300.000
index 690: buy 100.000000 units at price 2.320, current cash 99112.197, current litecoin 400.000
index 698: sell 100.000 units at price 2.829, current cash 98879.955, current litecoin 500.000
index 722: buy 100.000000 units at price 2.002, current cash 99162.561, current litecoin 400.000
index 723: buy 100.000000 units at price 2.250, current cash 98962.170, current litecoin 500.000
index 724: buy 100.000000 units at p

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
data.tail()

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in100MCnt,AdrBal1in10BCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1BCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalCnt,AdrBalNtv0.001Cnt,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,AdrBalNtv1Cnt,AdrBalNtv1KCnt,AdrBalNtv1MCnt,AdrBalUSD100Cnt,AdrBalUSD100KCnt,AdrBalUSD10Cnt,AdrBalUSD10KCnt,AdrBalUSD10MCnt,AdrBalUSD1Cnt,AdrBalUSD1KCnt,AdrBalUSD1MCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,BlkWghtMean,BlkWghtTot,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktEstUSD,CapMrktFFUSD,CapRealUSD,DiffLast,DiffMean,FeeByteMeanNtv,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,HashRate,IssContNtv,IssContPctAnn,IssContPctDay,IssContUSD,IssTotNtv,IssTotUSD,NDF,NVTAdj,NVTAdj90,NVTAdjFF,NVTAdjFF90,PriceBTC,PriceUSD,ROI1yr,ROI30d,ReferenceRate,ReferenceRateBTC,ReferenceRateETH,ReferenceRateEUR,ReferenceRateUSD,RevAllTimeUSD,RevNtv,RevUSD,SER,SplyAct10yr,SplyAct180d,SplyAct1d,SplyAct1yr,SplyAct2yr,SplyAct30d,SplyAct3yr,SplyAct4yr,SplyAct5yr,SplyAct7d,SplyAct90d,SplyActEver,SplyActPct1yr,SplyAdrBal1in100K,SplyAdrBal1in100M,SplyAdrBal1in10B,SplyAdrBal1in10K,SplyAdrBal1in10M,SplyAdrBal1in1B,SplyAdrBal1in1K,SplyAdrBal1in1M,SplyAdrBalNtv0.001,SplyAdrBalNtv0.01,SplyAdrBalNtv0.1,SplyAdrBalNtv1,SplyAdrBalNtv10,SplyAdrBalNtv100,SplyAdrBalNtv100K,SplyAdrBalNtv10K,SplyAdrBalNtv1K,SplyAdrBalNtv1M,SplyAdrBalUSD1,SplyAdrBalUSD10,SplyAdrBalUSD100,SplyAdrBalUSD100K,SplyAdrBalUSD10K,SplyAdrBalUSD10M,SplyAdrBalUSD1K,SplyAdrBalUSD1M,SplyAdrTop100,SplyAdrTop10Pct,SplyAdrTop1Pct,SplyCur,SplyExpFut10yr,SplyFF,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr,VtyDayRet180d,VtyDayRet30d,principal_market_price_usd,principal_market_usd,Volume,Annualized_DilutionRate,Total daily transaction volume,Velocity,The daily revenue of litecoin miners,The accumulated revenue of litecoin miners,P/E Ratio,Metcalfe,PM,<1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,5y~10y,>10y,One-year staking ratio,Two-year staking ratio,Three-year staking ratio,Four-year staking ratio,Five-year staking ratio,Ten-year staking ratio,Token Utility,PU Ratio,volatility30,volatility60,volatility90,volatility180,Token Utility_volatility,PU Ratio_volatility,state,cash,litecoin,holding,quantiles_low,quantiles_high,total,roi_gross,roi_daily,sharpe
2022-05-27,"381,746.00","5,276.00","917,515.00","2,466,172.00",750.00,"269,657.00","1,642,287.00",154.00,"44,327.00","6,008,588.00","3,456,171.00","2,242,489.00","1,535,833.00","32,973.00",125.00,"216,478.00",578.00,"785,973.00","3,821.00",5.00,"599,915.00","2,478.00","1,374,329.00","20,468.00",64.00,"2,027,571.00","148,061.00",403.00,"1,653,703,217.00",559.00,"118,518.93","355,894.24","199,300,777.00","2,214,766,684.12",0.54,0.51,"4,322,013,807.89","4,321,583,661.01","4,079,443,904.50","7,967,043,151.13","17,223,473.07","17,223,473.07",0.00,0.00,0.01,0.00,0.00,15.95,979.39,478.61,"6,987.50",3.62,0.01,"429,097.37","6,987.50","429,097.37",0.69,48.99,27.70,46.24,26.14,0.00,61.41,-68.30,-38.87,63.92,0.00,0.04,59.66,63.92,"2,102,426,515.04","7,003.45","430,076.76",0.03,"69,764,161.85","27,428,152.63","3,505,164.76","36,065,665.50","47,423,397.52","10,931,868.56","51,221,241.39","60,441,701.56","66,089,533.86","6,636,557.01","16,222,973.19","70,021,588.59",51.24,"56,654,963.21","70,159,322.78","70,376,198.65","48,321,577.50","68,669,761.34","70,357,321.45","36,528,320.87","63,931,714.35","70,378,747.99","70,374,145.97","70,348,195.25","70,043,503.15","68,214,258.77","62,966,229.96","34,133,397.62","46,849,826.09","55,416,647.45","9,831,405.39","70,371,611.03","70,327,964.78","69,822,380.98","53,765,978.58","61,441,285.23","27,005,627.87","67,369,955.24","44,713,771.75","31,633,397.61","69,823,912.93","64,849,720.66","70,380,462.82","81,645,309.38","66,430,410.18","104,865.00",1.21,"400,189.00","1,436,731.31","88,228,640.71",36.89,"2,265.34",0.03,2.09,85.07,0.05,0.07,0

In [15]:
figure =PUR.figure_signal(data=data)

In [16]:
figure = PUR.figure_portfolio(data=data)

In [17]:
figure = PUR.figure_returns(data=data)